# LAB 07 - Random Forest for Regression

In this lab we will be extending the previous lab about Decision trees and build a Regression model using Random Forest.

For simplicity, we will be using the same dataset as the previous lab (you can find it in ECLASS).

**IMPORTANT:** For this lab, if you haven't finished your code from last week's lab on Decision trees, you will have the option to use the sklearn implementation for a regression tree. However, this doesn't mean that you should skip the previous lab. This is just so that you don't get behind with the content and you don't spend all your time today working on the previous lab. 

In [1]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
from sklearn.model_selection import train_test_split

As mentioned before, use the Boston Housing data and prepare your train/val/test split as usual.

## Exercise 1 -- Bootstrap

Also known as [bagging](https://en.wikipedia.org/wiki/Bootstrap_aggregating), this technique consists of making several samples with replacement of the original data, using each of the samples to train an estimator, and then aggregating the predictions using the average (this is also a type of model ensemble).

In [2]:
def bootstrap(X, num_bags=10):
    """
    Given a dataset and a number of bags,
    sample the dataset with replacement.
    
    This function does not return a copy
    of the datapoints, but a list of indices
    with compatible dimensionality
    
    Parameters
    ----------
    X : ndarray
        A dataset
    num_bags : int, default 10
        The number of bags to create
    
    Returns
    -------
    list of ndarray
        The list contains `num_bags` integer one-dimensional ndarrays.
        Each of these contains the indices corresponding to the 
        sampled datapoints in `X`
    
    Notes
    -----
    * The number of datapoints in each bach will
      match the number of datapoints in the given
      dataset.
    * The
    """
    rng = np.random.default_rng(0) # you can change the seed, or use 0 to replicate my results
    # lista em que os elementos são arrays com os indices dos elementos que 
    list_bags = []
    n = X.shape[0]
    for i in range(num_bags):
        rand_idx = np.random.randint(0,n,size=n)
        list_bags.append(rand_idx)

    return list_bags
        
    # Your code here

In [3]:
rng = np.random.default_rng(0)
X_small = rng.random(size=(100,2))
bags = bootstrap(X_small)
bags[0]

array([29, 32, 26, 73, 23, 69, 53, 79, 19, 95, 41, 33,  4, 70, 64, 54,  9,
        1,  7, 62, 94, 29, 40, 38, 29, 46, 66, 86, 47, 74, 21, 68, 26,  8,
       94, 84, 27,  5, 58, 83, 79, 21, 16, 89, 89, 47, 98, 95, 48, 59, 75,
       39, 21,  2, 68,  5,  2, 85, 86, 17, 13, 78, 27, 60,  1, 44, 52, 26,
       22, 36, 14, 82, 68, 25, 80, 76, 90, 14, 46, 75, 47, 72, 65, 21, 37,
       84, 46,  1, 39, 71, 79, 29, 66, 78, 34, 15, 57, 71, 17, 43])

In [4]:
rng = np.random.default_rng(0)
X_small = rng.random(size=(100,2))
bags = bootstrap(X_small)
bags[0]

array([64, 13, 34, 84, 24, 98,  1, 87, 16, 87, 21, 70, 43, 41, 48, 42, 89,
       52, 86, 19, 78, 37, 56,  7, 75, 28, 23,  3, 57, 17, 37, 69, 73, 69,
       80, 94, 12, 33, 15, 43, 60,  0, 92, 71,  3, 63, 14, 75,  6, 47,  1,
       67,  2, 79, 55,  7, 13, 11, 43, 74, 10, 19, 26, 32, 93, 88, 47, 21,
       43, 28, 39, 87, 24, 66, 77, 83, 93, 63, 42, 11, 57, 83, 88, 40, 44,
       31, 99, 34, 84, 58, 88, 56, 41, 77, 58, 53, 69, 71, 74, 85])

## Exercise 2 -- Aggregation

The second part of bagging.

In [5]:
def aggregate_regression(preds):
    """
    Aggregate predictions by several estimators
    
    Parameters
    ----------
    preds : list of ndarray
        Predictions from multiple estimators.
        All ndarrays in this list should have the same
        dimensionality.
        
    Return
    ------
    ndarray
        The mean of the predictions
    """
    preds = np.array(preds).T

    return np.sum(preds,axis=1)/preds.shape[1]
    # Your code here

## Exercise 3 -- Random Forest for regression

Using the functions you implemented above, it is now time to put all of them together to train several decision trees and then ensemble them to output a single prediction. For the random forest, however, we need to select a subset of features at each split on the decision tree. 

For this part, you can use the sklearn implementation of Random forest for regression as your estimator for each set of features and bags. See below an example of how to do this, and always remember to check the necessary documentation when using an external function.

Some parameters you will have to set are: 
* num_features: number of features per estimator
* min_samples: min number of samples per leaf node
* max_depth: maximum depth of the decision tree (each estimator)
* num_estimators: number of decision trees you will create using each bag and random set of features

In [6]:
def regression_criterion(region):
    """
    Implements the sum of squared error criterion in a region
    
    Parameters
    ----------
    region : ndarray
        Array of shape (N,) containing the values of the target values 
        for N datapoints in the training set.
    
    Returns
    -------
    float
        The sum of squared error
        
    Note
    ----
    The error for an empty region should be infinity (use: float("inf"))
    This avoids creating empty regions
    """
    if len(region) < 1:
        return float("inf")
    mean = np.mean(region)
    target = region.reshape(-1,1)
    sse = np.sum(np.square(target-mean))
    
    return sse
    # your code here

In [7]:
def split_region(region, feature_index, tau):
    """
    Given a region, splits it based on the feature indicated by
    `feature_index`, the region will be split in two, where
    one side will contain all points with the feature with values 
    lower than `tau`, and the other split will contain the 
    remaining datapoints.
    
    Parameters
    ----------
    region : array of size (n_samples, n_features)
        a partition of the dataset (or the full dataset) to be split
    feature_index : int
        the index of the feature (column of the region array) used to make this partition
    tau : float
        The threshold used to make this partition
        
    Return
    ------
    left_partition : array
        indices of the datapoints in `region` where feature < `tau`
    right_partition : array
        indices of the datapoints in `region` where feature >= `tau` 
    """
    # your code here
    left_partition = []
    right_partition = []
    for i in range(region.shape[0]):
        if region[i, feature_index] < tau:
            left_partition.append(i)
        else:
            right_partition.append(i)
    
    return np.array(left_partition), np.array(right_partition)

In [8]:
def get_split(X, y):
    """
    Given a dataset (full or partial), splits it on the feature of that minimizes the sum of squared error
    
    Parameters
    ----------
    X : array (n_samples, n_features)
        features 
    y : array (n_samples, )
        labels
    
    Returns
    -------
    decision : dictionary
        keys are:
        * 'feature_index' -> an integer that indicates the feature (column) of `X` on which the data is split
        * 'tau' -> the threshold used to make the split
        * 'left_region' -> array of indices where the `feature_index`th feature of X is lower than `tau`
        * 'right_region' -> indices not in `low_region`
    """
    n = X.shape[0]
    m = X.shape[1]
    decision = {'feature_index': 0,
                 'tau':0,
                 'left_region':0,
                 'right_region':0}
    lowest_criterion = float("inf")
    for j in range(m):
        for i in range(n):
            tau = X[i,j]
            l, r = split_region(X, j, tau)
            if len(l) >= 1 and len(r) >= 1:
                sum_criterion = regression_criterion(y[l]) + regression_criterion(y[r])
            if len(r) < 1: 
                sum_criterion = regression_criterion(y[l]) + regression_criterion([])
            if len(l) < 1:
                sum_criterion = regression_criterion(y[r]) + regression_criterion([])
 
            if sum_criterion < lowest_criterion:
                lowest_criterion = sum_criterion    
                decision['feature_index'] = j
                decision['tau'] = tau
                decision['left_region'] = l
                decision['right_region'] = r
            
    return decision

In [9]:
def recursive_growth(node, min_samples, max_depth, current_depth, X, y):
    """
    Recursively grows a decision tree.
    
    Parameters
    ----------
    node : dictionary
        If the node is terminal, it contains only the "value" key, which determines the value to be used as a prediction.
        If the node is not terminal, the dictionary has the structure defined by `get_split`
    min_samples : int
        parameter for stopping criterion if a node has <= min_samples datapoints
    max_depth : int
        parameter for stopping criterion if a node belongs to this depth
    depth : int
        current distance from the root
    X : array (n_samples, n_features)
        features (full dataset)
    y : array (n_samples, )
        labels (full dataset)
    
    Notes
    -----
    To create a terminal node, a dictionary is created with a single "value" key, with a value that
    is the mean of the target variable
    
    'left' and 'right' keys are added to non-terminal nodes, which contain (possibly terminal) nodes 
    from higher levels of the tree:
    'left' corresponds to the 'left_region' key, and 'right' to the 'right_region' key
    """
    #verifica se é node terminal
    if 'left_region' not in node or 'right_region' not in node:
        return
    
    #pega os indices do datapoint
    left_indices = node['left_region']
    right_indices = node['right_region']
    
    #verifica criterio de parada pro node left
    if len(left_indices) <= min_samples or current_depth >= max_depth:
        node['left'] = {'value': np.mean(y[left_indices])}
    # faz partição do left 
    else:
        node['left'] = get_split(X[left_indices], y[left_indices])
        recursive_growth(node['left'], min_samples, max_depth, current_depth + 1, X, y)
    
    #verifica criterio de parada pro node right
    if len(right_indices) <= min_samples or current_depth >= max_depth:
        node['right'] = {'value': np.mean(y[right_indices])}
    # faz partição do right
    else:
        node['right'] = get_split(X[right_indices], y[right_indices])
        recursive_growth(node['right'], min_samples, max_depth, current_depth + 1, X, y)

In [10]:
def predict_sample(node, sample):
    """
    Makes a prediction based on the decision tree defined by `node`
    
    Parameters
    ----------
    node : dictionary
        A node created one of the methods above
    sample : array of size (n_features,)
        a sample datapoint
    """
    while 'value' not in node:
        if sample[node['feature_index']] >= node['tau']:
            node = node['right']
        else:
            node = node['left']
    
    return node['value']
    # your code here
        
def predict(node, X):
    """
    Makes a prediction based on the decision tree defined by `node`
    
    Parameters
    ----------
    node : dictionary
        A node created one of the methods above
    X : array of size (n_samples, n_features)
        n_samples predictions will be made
    """
    y_pred = []
    for x in X:
        pred = predict_sample(node,x)
        y_pred.append(pred)
    
    return np.array(y_pred)
    # your code here

In [11]:
boston = pd.read_csv("datasets/BostonHousing.txt")
X = boston.iloc[:,:13].to_numpy()
y = boston.iloc[:,13].to_numpy()
X_train, X_temp, y_train, y_temp = train_test_split(X,y, test_size = 0.2)
X_test, X_val, y_test, y_val = train_test_split(X_temp,y_temp, test_size = 0.5)

In [57]:
## your code goes here:
min_samples = 20
max_depth = 6
list_bags = bootstrap(X_train,num_bags=10)
preds = []
for bag in list_bags:
    root = get_split(X_train[bag],y_train[bag])
    recursive_growth(root,min_samples,max_depth,1, X_train[bag], y_train[bag])
    y_pred = predict(root,X_train[bag])
    print(y_pred.shape)
    preds.append(y_pred)
final_preds = aggregate_regression(preds)

(404,)
(404,)
(404,)
(404,)
(404,)
(404,)
(404,)
(404,)
(404,)
(404,)


In [58]:
final_preds.shape

(404,)

In [59]:
rmse = np.sqrt(np.mean(np.square(y_train-final_preds)))
rmse

9.288186709361407